# Mixed-Logit Model Example

In [1]:
import autograd.numpy as np
from autograd import grad
from autograd import hessian
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

/Users/limingyi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/limingyi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


- Utility function
$$
U_{nj} = \beta_0 \times quality_{j} - \beta_1 \times\frac{price_j}{income_n} + \beta_2 \times popularity_j + \epsilon_{nj}
$$
where $\beta_0 \sim LN(0.8,0.7); \beta_1 \sim LN(0.5,0.8); \beta_2 \sim N(0.6,0.5)$

In [11]:
# Data Generation Process


# Number of consumers
N = 5000

# Number of alternatives
K = 4

# Generate goods characteristics
quality = np.random.normal(scale = 2, loc = 0,size = K)
price = np.random.normal(scale = 2, loc = 0,size = K)
popularity = np.random.normal(scale =1, loc = 0,size = K)
goods = pd.DataFrame(np.concatenate([np.arange(K).reshape(-1,1),quality.reshape(-1,1),price.reshape(-1,1),popularity.reshape(-1,1)],axis=1),columns = ['gid','quality','price','popularity'])




# Generate True underlying parameters following specific distributions
true_params = [.8,.7,.5,.8,.6,.5]

b0 = np.random.lognormal(mean = true_params[0],sigma = true_params[1],size = N)
b1 = np.random.lognormal(mean = true_params[2],sigma = true_params[3],size = N)
b2 = np.random.normal(loc = true_params[4],scale = true_params[5], size = N)
b = np.concatenate([b0.reshape(-1,1),b1.reshape(-1,1),b2.reshape(-1,1)],axis=1)

# Generate consumer characteristics
income = np.random.uniform(low= 2, high = 6,size = N)
user = pd.DataFrame(np.concatenate([np.arange(N).reshape(-1,1),b, income.reshape(-1,1)],axis=1),columns = ['uid','b0','b1','b2','income'])

# error term, Gumbel(0,1)
e = np.random.gumbel(size = N*K)


data = user.merge(goods,how = 'cross')

In [12]:
# Generate N-K matrix

data['e'] = e
data['p/i'] = data['price']/data['income']
data['u'] = data['b0']* data['quality'] - data['b1']* data['p/i'] + data['b2']* data['popularity'] + data['e']
data['max_u'] = data.groupby('uid')['u'].transform('max')

# Find consumer's choices based on utiilty maximization
y = data.query('max_u == u')[['uid','gid']].rename({'gid':'y'},axis=1)
data = data.merge(y)

# Observed choices, one-hot vector, shape = (N *K,)
data['choice'] = 0
data.loc[data.query('y== gid').index,'choice']=1
choice = np.array(data['choice'])


# Observed X
X = data[['quality','p/i','popularity']].values


In [13]:
Counter(data.query('choice==1')['y'])

Counter({0.0: 2751, 3.0: 1988, 1.0: 125, 2.0: 136})

### MSL

In [14]:
# Observed X
X = data[['quality','p/i','popularity']].values

In [15]:
# Change price to -price, to ensure its coefficient is positive
X[:,1] = -X[:,1]
X = X.reshape(N,4,-1)

In [16]:
def get_prob(params):
    """
    Params: params-----> parameters in the model
    Return: Log-likelihood function given parameters
    """
    # Since sigma has to be positive, I use exponential form 
    params = np.exp(params)
    b0 = np.random.lognormal(mean = params[0],sigma = params[1],size = [R,N,1])
    b1 = np.random.lognormal(mean = params[2],sigma = params[3],size = [R,N,1])
    b2 = np.random.normal(loc = params[4],scale = params[5], size = [R,N,1])
    b = np.concatenate([b0,b1,b2],axis=2)
    V = X@b.reshape(R,N,3,1)
    pr = np.exp(V)
    p = pr/pr.sum(axis=2,keepdims=True)
    p_mean = p.mean(axis=0)
    logp = np.log(p_mean)
    outcome = data['choice'].values
    prob = -logp.reshape(N,1,4)@(outcome.reshape(N,K,1))
    return prob.sum()

In [17]:
def get_true_prob(params):
    """
    Params: params-----> True parameters in the model
    Return: Simulated Log-likelihood function under true parameters
    """
    b0 = np.random.lognormal(mean = params[0],sigma = params[1],size = [R,N,1])
    b1 = np.random.lognormal(mean = params[2],sigma = params[3],size = [R,N,1])
    b2 = np.random.normal(loc = params[4],scale = params[5], size = [R,N,1])
    b = np.concatenate([b0,b1,b2],axis=2)
    V = X@b.reshape(R,N,3,1)
    pr = np.exp(V)
    p = pr/pr.sum(axis=2,keepdims=True)
    p_mean = p.mean(axis=0)
    logp = np.log(p_mean)
    outcome = data['choice'].values
    prob = -logp.reshape(N,1,4)@(outcome.reshape(N,K,1))
    return prob.sum()

In [34]:
# Repeat 100 times 
R = 100
params = np.random.normal(size=6,loc=-1,scale= .5)
# params = np.random.uniform(size=6,low=.1,high=.9)

### maximize simulated log-likelihood function
res = opt.minimize(get_prob,x0=params,method= 'L-BFGS-B')
print(res)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 4391.878399151793
        x: [-7.646e-01 -9.887e-01 -8.833e-01 -9.568e-01 -7.152e-01
            -1.277e+00]
      nit: 3
      jac: [-2.739e+08 -1.715e+08 -2.263e+08 -4.271e+07 -3.576e+08
             1.351e+08]
     nfev: 217
     njev: 31
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>


In [35]:
print('Initial Parameters = ',np.exp(params))

Initial Parameters =  [0.46552444 0.37205835 0.41343528 0.38410375 0.48911697 0.27899909]


In [36]:

print('Estimated Parameters = ',np.exp(res.x))
print('True Parameters = ',true_params)
print('Simulated log-likelihood  = ',-res.fun)
print('True log-likelihood = ',-get_true_prob(true_params))

Estimated Parameters =  [0.46552444 0.37205835 0.41343528 0.38410375 0.48911696 0.27899909]
True Parameters =  [0.8, 0.7, 0.5, 0.8, 0.6, 0.5]
Simulated log-likelihood  =  -4391.878399151793
True log-likelihood =  -4365.710302319214


### MOM

In [25]:
def m_condition(params):
    """
    Params: params-----> True parameters in the model
    Return: sum of the quadratic form of error term
    """
    params = np.exp(params)
    b0 = np.random.lognormal(mean = params[0],sigma = params[1],size = [R,N,1])
    b1 = np.random.lognormal(mean = params[2],sigma = params[3],size = [R,N,1])
    b2 = np.random.normal(loc = params[4],scale = params[5], size = [R,N,1])
    b = np.concatenate([b0,b1,b2],axis=2)
    V = X@b.reshape(R,N,3,1)
    pr = np.exp(V)
    p = pr/pr.sum(axis=2,keepdims=True)
    p_mean = p.mean(axis=0)
    outcome = data['choice'].values
    error = choice.reshape(N,K,-1) - p_mean
    res = (X.reshape(N,3,4)@error)**2
    return res.sum()

In [37]:
# Repeat R times 
### minimize simulated GMM error
res = opt.minimize(m_condition,x0=params,method= 'L-BFGS-B')
print(res)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 14791.561546050478
        x: [-7.646e-01 -9.887e-01 -8.833e-01 -9.568e-01 -7.152e-01
            -1.277e+00]
      nit: 5
      jac: [-2.510e+09 -1.127e+09 -3.085e+09  1.057e+08 -2.149e+08
            -6.668e+08]
     nfev: 252
     njev: 36
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>


In [38]:
print('Initial Parameters = ',np.exp(params))
print('Estimated Parameters = ',np.exp(res.x))
print('True Parameters = ',true_params)
print('Simulated Error  = ',res.fun)
print('True Error = ',m_condition(np.log(true_params)))

Initial Parameters =  [0.46552444 0.37205835 0.41343528 0.38410375 0.48911697 0.27899909]
Estimated Parameters =  [0.46552449 0.3720583  0.4134353  0.38410371 0.48911692 0.27899907]
True Parameters =  [0.8, 0.7, 0.5, 0.8, 0.6, 0.5]
Simulated Error  =  14791.561546050478
True Error =  14680.54344096777
